In [8]:
import time
import os
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import requests
from IPython.display import display, HTML
from IPython.display import Image
from urllib.parse import urljoin


In [3]:
df = pd.read_csv('final_corpus.csv')

In [11]:
def slow_scroll(url, close_driver=True):
    
    driver = webdriver.Chrome()

    # Open the webpage using the browser
    driver.get(url)


    SCROLL_PAUSE_TIME = 0.05

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    scroll_dist = 0
    while True:
        scroll_dist += 100
        # Scroll down to bottom
        driver.execute_script(f"window.scrollTo(0, {scroll_dist});")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if scroll_dist < new_height:
            continue

        if new_height == last_height:
            break
        last_height = new_height

    # Wait for the page to fully load (you might need to adjust the time)
    driver.implicitly_wait(10)  # Wait for up to 10 seconds

    # Get the page source after JavaScript execution
    page_source = driver.page_source
    
    if close_driver:
        driver.quit()
    return page_source

In [89]:
def parse_wapo(page_source, outdir, prefix):
    
    soup = BeautifulSoup(page_source, "html.parser")
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)
        
        
    article_text = []
    title = soup.find('h1')
    title_text = title.get_text()
    article_text.append(title_text)
    
    # Find all txt elements within the article content
    article = soup.find('article')
    txt_elements = article.find_all('p')

    # Display the images inline in the notebook
    for txt in txt_elements:
        text = txt.get_text()
        article_text.append(text)
        
    
        
    article_text = '\n'.join(article_text)
    
    text_filename = f'{prefix:03d}_text.txt'
    
    with open(os.path.join(outdir, text_filename), 'w') as file:
        file.write(article_text)
        
    images = []
    
    # Find all img elements within the article content
    img_elements = article.find_all("img")

    # Grab url
    for img in img_elements:
        img_src = img.get("src")
        
        if img_src is None:
            if img.get('srcset') is None:
                continue
            else:
                img_src = img.get('srcset')
                img_src_pair = [pair.strip() for pair in img_src.split(',')][0]
                img_src = img_src_pair.split(' ')[0]

        if img_src[:4]=='http':
            images.append(img_src)
        else:
            absolute_img_src = urljoin(url, img_src)
            images.append(absolute_img_src)
        
    for i,img_url in enumerate(images):
        print(f'request image {prefix}_{i}')
        response = requests.get(img_url, stream=True)
        
        try:
            image_obj = Image.open(response.raw)
            
            filename = f'{prefix:03d}_{i:02d}.png'

            image_obj.save(os.path.join(outdir, filename))
        except:
            continue
        
        
    
    
    

In [ ]:
start_idx = 17
for r, row in df.iterrows():
    
    if r < start_idx:
        continue
    
    outlet = row['Outlet']
    url = row['Article']
    
    if outlet == 'WaPo':
        
        page_source = slow_scroll(url, close_driver=False)
        print(f'scrolled {r}')
        
        parse_wapo(page_source,'./results/WaPo',r)


scrolled 17
request image 17_0
request image 17_1
request image 17_2
request image 17_3
request image 17_4
request image 17_5
request image 17_6
request image 17_7
request image 17_8
request image 17_9
request image 17_10
request image 17_11
request image 17_12
request image 17_13
request image 17_14
request image 17_15
request image 17_16
scrolled 18
request image 18_0
request image 18_1
request image 18_2
request image 18_3
request image 18_4
request image 18_5
request image 18_6
request image 18_7
request image 18_8
request image 18_9
request image 18_10
request image 18_11
request image 18_12
request image 18_13
request image 18_14
scrolled 19
request image 19_0
request image 19_1
request image 19_2
request image 19_3
request image 19_4
request image 19_5
request image 19_6
request image 19_7
request image 19_8
request image 19_9
request image 19_10
request image 19_11
request image 19_12
scrolled 20
request image 20_0
request image 20_1
request image 20_2
scrolled 22
request image

In [15]:
from PIL import Image

In [20]:
parse_wapo(page_source,1,2)

TypeError: 'NoneType' object is not subscriptable

In [61]:
page_source

'<html lang="en" class="story layout_article-single-column pb_actual_layout_article-single-column rendering-context-www outputtype_default-article no-mobile desktop no-ie chrome chrome115 business-section 2020-section 04-section 17-section last-year-we-searched-google-how-tie-tie-now-were-using-it-find-toilet-paper-section js" id="index-page"><head> <iframe src="javascript:false" title="" role="presentation" style="width: 0px; height: 0px; border: 0px; display: none;"></iframe><script type="text/javascript" async="" src="https://aax-us-east.amazon-adsystem.com/e/dtb/admj?b=JATkOJFyZFhBqcAdzoifzhEAAAGJ8KKFwAEAAAvhAQBhcHNfdHhuX2JpZDEgICBhcHNfdHhuX2ltcDEgICDND2sX&amp;rnd=9577314367611691959330805&amp;pp=1n9ccn4&amp;p=18rfke8&amp;crid=451831027"></script><script type="text/javascript" async="" src="https://www.gstatic.com/recaptcha/releases/3kTz7WGoZLQTivI-amNftGZO/recaptcha__en.js" crossorigin="anonymous" integrity="sha384-bPGPp3F76oplmIWtwkyKzLqEL/SHRM2WY01hYjmEh4dQ/PAlqx+Z/KgDxg33uhOa">

In [87]:
    soup = BeautifulSoup(page_source, "html.parser")

    images = []
    
    # Find all img elements within the article content
    img_elements = soup.find_all("img")

    # Display the images inline in the notebook
    for img in img_elements:
        img_src = img.get("src")
        #print(img_src)
        
        if img_src is None:
            if img.get('srcset') is None:
                #print(img)
                continue
            else:
                #import pdb; pdb.set_trace()
                img_src = img.get('srcset')
                #print(img_src)

        if img_src[:4]=='http':
            images.append(img_src)
        else:
            
            absolute_img_src = urljoin(url, img_src)
            images.append(absolute_img_src)
            
        
    for i,img_url in enumerate(images):
        print(img_url)
        
        #response = requests.get(img_url, stream=True)
        
        try:
            image_obj = Image.open(response.raw)

            #image_obj.save(f'./test_img_{i}.png')
        except:
            continue

> /tmp/ipykernel_141/3437551169.py(19)<module>()
     17         else:
     18             import pdb; pdb.set_trace()
---> 19             img_src = img.get('srcset')
     20             #print(img_src)
     21 

ipdb> img
<img alt="" class="w-100 mw-100 h-auto" decoding="async" height="450" sizes="(max-width: 440px) 440px,(max-width: 600px) 691px,(max-width: 768px) 691px,(min-width: 769px) and (max-width: 1023px) 960px,(min-width: 1024px) and (max-width: 1299px) 530px,(min-width: 1300px) and (max-width: 1439px) 691px,(min-width: 1440px) 916px,440px" srcset="https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 400w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 540w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 691w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 767w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_15873

In [76]:
img_elements

[<img alt="" class="wpds-c-dgBqAZ" src="https://www.washingtonpost.com/wp-apps/imrs.php?src=https://s3.amazonaws.com/arc-authors/washpost/f51547ea-6514-4237-a4c1-be32b4b634c2.png&amp;w=196&amp;h=196"/>,
 <img alt="" class="w-100 mw-100 h-auto" decoding="async" height="450" sizes="(max-width: 440px) 440px,(max-width: 600px) 691px,(max-width: 768px) 691px,(min-width: 769px) and (max-width: 1023px) 960px,(min-width: 1024px) and (max-width: 1299px) 530px,(min-width: 1300px) and (max-width: 1439px) 691px,(min-width: 1440px) 916px,440px" srcset="https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 400w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 540w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 691w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 767w,https://palomaimages.washingtonpost.com/prod/NBCWSJopentooquick_tstmp_1587359360.jpg 916w,http

In [30]:
response.raw

In [33]:
img_url 

'https://www.washingtonpost.com/resizer/ZP8LYU-5H-pq2ocspyx_XL3s2QA=/1200x630/filters:focal(759x427:1541x867):quality(100)/arc-anglerfish-washpost-prod-washpost/XQB62S46VJFZXP5FDFNABMPDEQ.jpg?c=5bb2f1c2d744a616473a8b543dba4ae8d5d385a7-1586371551'

In [62]:
url

'https://www.washingtonpost.com/business/2020/04/17/last-year-we-searched-google-how-tie-tie-now-were-using-it-find-toilet-paper/?utm_source=rss&utm_medium=referral&utm_campaign=wp_homepage'

In [48]:
soup = BeautifulSoup(page_source, "html.parser")

In [63]:
txt_elements = soup.find_all('p')

In [65]:
[txt.get_text() for txt in txt_elements]

['clockThis article was published more than\xa03 years ago',
 'During the daily White House coronavirus press briefing early last week, President Trump praised the country’s testing regimen for the virus.',
 '“We have now done 1,790,000 tests nationwide,” Trump said. “That’s more than any other country in the world, hence we have more cases. And that number is growing by nearly 125,000 people per day. Think of that one. So it’s growing by 125,000 people per day.”',
 '“I told you about South Korea,” he added. “This is a, you know, vastly faster — and we also have, they say, the most accurate of tests.”',
 'The mention of South Korea was an attempt to contrast the United States’ testing with testing in a country that has been seen as having contained the coronavirus through broad testing. The virus was detected in both the United States and South Korea at the same time, but, for a variety of reasons, including testing, South Korea better contained the virus’s spread within its borders.',

In [66]:
soup = BeautifulSoup(page_source, "html.parser")

In [67]:
article = soup.find('article')

In [69]:
article.find_all('p')

[<p data-elm-loc="0">April 2019 was a simpler time. Americans wanted to know what a black hole was, who was playing in the Final Four and how many episodes of “Game of Thrones” there were.</p>,
 <p data-elm-loc="1">This April, we suddenly need to know how to make masks, and millions more are searching for <a href="https://www.washingtonpost.com/business/2020/04/16/faq-unemployment-covid-19/" target="_blank">how to file for unemployment</a>. There are some constants in life, though: We still need help nailing the perfect boiled egg.</p>,
 <p data-elm-loc="2">We compared top Google searches during the same weeks in 2019 and 2020 to see what has and hasn’t changed.</p>,
 <p class="chatter">Most common Google searches beginning with “how to” from April 5 to 11.</p>,
 <p class="g-pstyle0">2019</p>,
 <p class="g-pstyle0">2020</p>,
 <p class="g-pstyle1">Train Your Dragon 3</p>,
 <p class="g-pstyle1">make a mask</p>,
 <p class="g-pstyle1">make hand sanitizer</p>,
 <p class="g-pstyle1">take a s

In [74]:
page_source

'<html lang="en" style="overflow: visible;"><head><meta charset="utf-8"><link rel="icon" href="https://www.washingtonpost.com/favicon.svg" type="image/svg+xml"><link rel="icon" href="https://www.washingtonpost.com/touch-icon-iphone.png" type="image/png"><link rel="apple-touch-icon" href="https://www.washingtonpost.com/touch-icon-iphone.png"><link rel="apple-touch-icon" sizes="152x152" href="https://www.washingtonpost.com/touch-icon-ipad.png"><link rel="apple-touch-icon" sizes="167x167" href="https://www.washingtonpost.com/touch-icon-ipad-retina.png"><link rel="apple-touch-icon" sizes="180x180" href="https://www.washingtonpost.com/touch-icon-iphone-retina.png"><title>The Finance 202: Most Americans aren\'t ready yet to reopen the economy - The Washington Post</title><meta name="description" content="Three-fourths think the economic shutdown will last months, not weeks, new poll finds."><meta property="og:site_name" content="Washington Post"><meta property="og:type" content="article"><me